In [ ]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import h5py
from matplotlib.colors import TwoSlopeNorm

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])
    
# ------------------ LOADERS ------------------
def load_fof_gas(basepath, snap, halo_id):
    """
    Gas bound to a FoF halo via Subfind catalogs:
    returns positions [kpc], masses [Msun], and a suggested center [kpc].
    """
    # FoF group catalog entry
    grp = il.groupcat.loadSingle(basepath, snap, haloID=halo_id)
    center_ckpch = grp['GroupPos']     # catalog center (ckpc/h)

    # Particle data for that FoF
    gas = il.snapshot.loadHalo(
        basepath, snap, halo_id, partType='gas',
        fields=['Coordinates','Masses']
    )
    coords = gas['Coordinates']        # ckpc/h
    masses = gas['Masses']             # 1e10 Msun/h

    # Convert to kpc and Msun
    coords_kpc  = coords / h
    masses_Msun = masses * (1e10 / h)
    center_kpc  = center_ckpch / h
    box_kpc     = boxsize / h

    # Recenter with periodic wrapping so the halo sits near (0,0,0)
    xyz = coords_kpc - center_kpc
    xyz = (xyz + 0.5*box_kpc) % box_kpc - 0.5*box_kpc

    return xyz, masses_Msun, center_kpc

def load_subhalo_gas(basepath, snap, sub_id):
    """
    Gas bound to a *subhalo* (galaxy) via Subfind catalogs.
    """
    sub = il.groupcat.loadSingle(basepath, snap, subhaloID=sub_id)
    center_ckpch = sub['SubhaloPos']

    gas = il.snapshot.loadSubhalo(
        basepath, snap, sub_id, partType='gas',
        fields=['Coordinates','Masses']
    )
    coords = gas['Coordinates']
    masses = gas['Masses']

    coords_kpc  = coords / h
    masses_Msun = masses * (1e10 / h)
    center_kpc  = center_ckpch / h
    box_kpc     = boxsize / h

    xyz = coords_kpc - center_kpc
    xyz = (xyz + 0.5*box_kpc) % box_kpc - 0.5*box_kpc

    return xyz, masses_Msun, center_kpc

In [ ]:
basepath = "/gpfs/gibbs/pi/nagai/IllustrisTNG_data/TNG50-1/output"
snap     = 99
sub_id  = mw_id[0]

sub = il.groupcat.loadSingle(basepath, snap, subhaloID=sub_id)
center_ckpch = sub['SubhaloPos']

gas = il.snapshot.loadSubhalo(
    basepath, snap, sub_id, partType='gas',
    fields=['Coordinates','Masses','Velocities','temp']
)
gas